In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
import os

import torch

In [7]:
!pip install ultralytics

from ultralytics import YOLO

In [8]:
#img = Image.open('/kaggle/input/myocardits-dataset-cleaned-complete/Normal/Individuo_01/series0001-Body/img0001--9.jpg')

In [ ]:
num_anotacoes = 0
img_por_serie = 0
num_anot_ind = []

from PIL import Image, ImageOps

def adjust_bbox_to_square(x_min, y_min, x_max, y_max, width, height):
    # Ajusta a bounding box para ser quadrada
    bbox_width = x_max - x_min
    bbox_height = y_max - y_min
    max_side = max(bbox_width, bbox_height)
    
    # Calcula o centro da bounding box
    x_center = x_min + bbox_width / 2
    y_center = y_min + bbox_height / 2
    
    # Calcula as novas coordenadas da bounding box quadrada
    x_min_new = int(x_center - max_side / 2)
    y_min_new = int(y_center - max_side / 2)
    x_max_new = int(x_center + max_side / 2)
    y_max_new = int(y_center + max_side / 2)
    
    # Garante que a bounding box não ultrapasse os limites da imagem
    x_min_new = max(0, x_min_new)
    y_min_new = max(0, y_min_new)
    x_max_new = min(width, x_max_new)
    y_max_new = min(height, y_max_new)
    
    return x_min_new, y_min_new, x_max_new, y_max_new

def save_cropped_images(image, annotations, output_dir, filename):
    global img_por_serie
    
    width, height = image.size
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    for i, annotation in enumerate(annotations):
        parts = annotation.split()
        class_id = int(parts[0])
        x_center = float(parts[1]) * width
        y_center = float(parts[2]) * height
        bbox_width = float(parts[3]) * width
        bbox_height = float(parts[4]) * height

        x_min = int(x_center - bbox_width / 2)
        y_min = int(y_center - bbox_height / 2)
        x_max = int(x_center + bbox_width / 2)
        y_max = int(y_center + bbox_height / 2)

        # Ajustar a bounding box para ser quadrada
        x_min, y_min, x_max, y_max = adjust_bbox_to_square(x_min, y_min, x_max, y_max, width, height)
        
        # Recortar a imagem
        cropped_image = image.crop((x_min, y_min, x_max, y_max))
        
        # Criar uma imagem quadrada com fundo preto
        max_side = max(x_max - x_min, y_max - y_min)
        square_image = Image.new("RGB", (max_side, max_side), (0, 0, 0))
        
        # Colar a imagem recortada no centro da imagem quadrada
        paste_x = (max_side - cropped_image.width) // 2
        paste_y = (max_side - cropped_image.height) // 2
        square_image.paste(cropped_image, (paste_x, paste_y))
        
        # Salvar a imagem quadrada
        cropped_image_path = os.path.join(output_dir, f"{filename[:-4]}.jpg")
        square_image.save(cropped_image_path)
        print(f"Imagem recortada salva em {cropped_image_path}")
        
        img_por_serie += 1

# Função de anotação e recorte
def annotate_and_save_cropped_images(directory, output_dir, annotations_dir, model):
    global num_anotacoes
    
    if not os.path.exists(directory):
        #print(f"Diretório de imagens não encontrado: {directory}")
        return
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
        
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory, filename)
            print(f"Processando imagem: {filename}")

            # Carregar imagem
            image = Image.open(image_path).convert("RGB")
            width, height = image.size
            
            # Realizar inferência com o YOLOv8
            results = model(image_path)
            
            # Processar as anotações
            for result in results:
                annotations = result.boxes.xyxy.cpu().numpy()  # Bounding boxes em formato xyxy
                confidences = result.boxes.conf.cpu().numpy()  # Confiança das detecções
                classes = result.boxes.cls.cpu().numpy()  # Classes preditas
                
                yolo_annotations = []
                
                if len(annotations) == 0:
                    print(f"Nenhuma anotação encontrada para {filename}")
                    continue
                
                # Iterar pelas detecções
                for i, (bbox, confidence, cls) in enumerate(zip(annotations, confidences, classes)):
                    x_min, y_min, x_max, y_max = map(int, bbox)
                    x_center = (x_min + x_max) / 2
                    y_center = (y_min + y_max) / 2
                    bbox_width = x_max - x_min
                    bbox_height = y_max - y_min
                    
                    # Formatar anotação no estilo YOLO
                    yolo_annotations.append(f"{int(cls)} {x_center / width} {y_center / height} {bbox_width / width} {bbox_height / height}")
                    
                    # Salvar as imagens recortadas (se necessário)
                    save_cropped_images(image, yolo_annotations, output_dir, filename)
                
                # Salvar as anotações no formato YOLO
                annotation_path = os.path.join(annotations_dir, f"{os.path.splitext(filename)[0]}.txt")
                with open(annotation_path, "w") as annotation_file:
                    annotation_file.write("\n".join(yolo_annotations))
                        
                print(f"Anotações YOLO salvas para {filename}")
                num_anotacoes += 1

# Carregar o modelo YOLOv8
model_path = '/kaggle/input/heart-detection-model-group0/pytorch/default/2/weights/best.pt'
model = YOLO(model_path)

# DIRETÓRIOS
DATASET_PATH = '/kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6'
CLASS_TYPE = ['Normal', 'Sick']

INDIVIDUOS_NORMAL = [f'Individuo_{i:02}' for i in range(1, 17)]
INDIVIDUOS_SICK = [f'Individuo_{i:02}' for i in range(17, 48)]
#SHORT_LONG = ['1-short', '2-long']
SERIES = [f'series{i:04}-Body' for i in range(1, 126)]

for Class in CLASS_TYPE:
    if Class == 'Normal':
        INDIVIDUOS = INDIVIDUOS_NORMAL
    else:
        INDIVIDUOS = INDIVIDUOS_SICK
        
    for individuo in INDIVIDUOS:
        for serie in SERIES:
            image_directory = f'{DATASET_PATH}/{Class}/{individuo}/{serie}'
            print(f"Processando diretório: {image_directory}")

            # Diretório de saída para salvar as imagens recortadas
            output_directory = f"/kaggle/working/cropped_images/{Class}/{individuo}/{serie}"
            os.makedirs(output_directory, exist_ok=True)

            annotations_directory = f"/kaggle/working/annotations/{Class}/{individuo}/{serie}"
            os.makedirs(annotations_directory, exist_ok=True)

            annotate_and_save_cropped_images(image_directory, output_directory, annotations_directory, model)

            plt.close()
            img_por_serie = 0

        print(f'Número total de anotações do {individuo}: {num_anotacoes}')
        num_anot_ind.append(num_anotacoes)
        num_anotacoes = 0

print('Número de anotações por individuo:')
i = 1
for num in num_anot_ind:
    print(f'Individuo {i:02}: {num} anotações')
    i+=1

Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0001-Body
Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0002-Body
Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0003-Body
Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0004-Body
Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0005-Body
Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0006-Body
Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0007-Body
Processando diretório: /kaggle/input/myocardits-dataset-clustering-groups/GRUPO_6/Normal/Individuo_01/series0008-Body
Processando diretório: /kaggle/input/myocardits-dataset-

In [ ]:
# Zipando pasta com resultados
import zipfile

def compactar_pasta(pasta, nome_arquivo_zip):
    with zipfile.ZipFile(nome_arquivo_zip, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, _, files in os.walk(pasta):
            for arquivo in files:
                caminho_completo = os.path.join(root, arquivo)
                zip_file.write(caminho_completo, os.path.relpath(caminho_completo, pasta))

nome_arquivo_zip = f'heart-detection.zip'
compactar_pasta('/kaggle/working/', nome_arquivo_zip)